<h1 style='text-align:center'>Intro Data Science<h1>

In [ ]:
import numpy as np # Para manipulação numérica
import pandas as pd # Para manipulação do dataset
import matplotlib.pyplot as plt # Visualização de gráficos
import seaborn as sns # Visualização de gráficos

Vamos explorar o dataset sobre Brasil. Nele temos informação geográficas como:
* Popoulação
* IDH
* PIB *Per capta*
* etc

Alêm dos dados geográficos, também estão presentes no dataset alguns dados mais variados como:
* Presença de Uber
* Quantidade de McDonald's
* Quantidade de Wal-Mart
* etc


Vamos começar pela leitura do dataset

In [ ]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

credentials_1 = {
    'IAM_SERVICE_ID': 'Let me take you down',
    'IBM_API_KEY_ID': 'Cause I`m going to Strawberry Fields',
    'ENDPOINT': 'Nothing is real',
    'IBM_AUTH_ENDPOINT': 'And nothing to get hung about',
    'BUCKET': 'Strawberry Fields forever',
    'FILE': 'BRAZIL_CITIES.csv'
}

client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
    ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials_1['ENDPOINT'])


with open('BRAZIL_CITIES.csv', 'wb') as data:
    client.download_fileobj(credentials_1['BUCKET'], 'BRAZIL_CITIES.csv', data)

In [ ]:
df_01 = pd.read_csv('./BRAZIL_CITIES.csv', sep=';', decimal=',')

Com dataset disponível para uso, vamos verificar o conteûdo do mesmo.

In [ ]:
df_01.head()

Os dados parecem corretos, mas vamos dar uma olhanda mais a fundo para nos certificamos de que os tipos estão corretos.

In [ ]:
df_01.info()

Olhando para o dataset e verificandos os tipos dados que nos foi entregue, fica evidente que antes de iniciarmos qualquer trabalho de analise devemos corrigir e limpar imperfeições do dataset.

In [ ]:
def corrigeTipo(d, c, t):
    return d[[c]].astype(t)


df_01[['IDHM']] = corrigeTipo(df_01, 'IDHM', 'float')
df_01[['IDHM_Renda']] = corrigeTipo(df_01, 'IDHM_Renda', 'float')
df_01[['IDHM_Longevidade']] = corrigeTipo(df_01, 'IDHM_Longevidade', 'float')
df_01[['IDHM_Educacao']] = corrigeTipo(df_01, 'IDHM_Educacao', 'float')
df_01[['LONG']] = corrigeTipo(df_01, 'LONG', 'float')
df_01[['LAT']] = corrigeTipo(df_01, 'LAT', 'float')
df_01[['ALT']] = corrigeTipo(df_01, 'ALT', 'float')
df_01[['GVA_AGROPEC']] = corrigeTipo(df_01, 'GVA_AGROPEC', 'float')
df_01[['GVA_INDUSTRY']] = corrigeTipo(df_01, 'GVA_INDUSTRY', 'float')
df_01[['GVA_SERVICES']] = corrigeTipo(df_01, 'GVA_SERVICES', 'float')
df_01[['GVA_PUBLIC']] = corrigeTipo(df_01, 'GVA_PUBLIC', 'float')
df_01[[' GVA_TOTAL ']] = corrigeTipo(df_01, ' GVA_TOTAL ', 'float')
df_01[['TAXES']] = corrigeTipo(df_01, 'TAXES', 'float')
df_01[['GDP']] = corrigeTipo(df_01, 'GDP', 'float')
df_01[['POP_GDP']] = corrigeTipo(df_01, 'POP_GDP', 'float')
df_01[['GDP_CAPITA']] = corrigeTipo(df_01, 'GDP_CAPITA', 'float')

Para corrigir à área teremos que ser uma pouco mais criativos.

In [ ]:
def castStrNumber(s):
    if len(s) == 2:
        return s[0]+s[1]
    elif len(s) > 2:
        return s[0]+castStrNumber(s[1:len(s)])
    else:
        return s[0]

area = []
for i,j in df_01[['AREA']].iterrows():
    if type(j[0]) == str:
        area.append(castStrNumber(j[0].split(',')))
    else:
        area.append(j[0])
    
df_01[['AREA']] = np.array(area)
df_01[['AREA']] = df_01[['AREA']].astype(float)

In [ ]:
df_01.fillna(0., inplace=True)

In [ ]:
df_01.info()

Agora parece tudo correto com relação ao tipo dos dados, então vamos explorar um pouco mais do dataset

In [ ]:
df_01.describe()

In [ ]:
df_01.count()

In [ ]:
df_01["STATE"].value_counts().shape

Com dataset limpo e adequado, agora podemos começar a explorar algumas métricas. Vamos verificar a quintidade de municípios por estado.

In [ ]:
df_01["STATE"].value_counts()

In [ ]:
df_01[['STATE', 'CITY']].groupby('STATE').count()

Agora vamos começar a explorar um pouco mais gerar visualizações para os nossos dados.

In [ ]:
maskLat = df_01["LONG"] != 0
maskLong = df_01["LAT"] != 0 

x = df_01[maskLat&maskLong]["LONG"]
y = df_01[maskLat&maskLong]["LAT"]
z = df_01[maskLat&maskLong]["IDHM"]

plt.figure(figsize=(10,10))
plt.title("Longitude vs Latitude - Localização das cidades do Brasil")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.scatter(x, y, s=z, alpha=1)
plt.show()

In [ ]:
x = df_01[maskLat&maskLong]["LONG"]
y = df_01[maskLat&maskLong]["LAT"]
z = df_01[maskLat&maskLong]["ESTIMATED_POP"]

plt.figure(figsize=(10,10))
plt.title("Longitude vs Latitude - Localização da população do Brasil")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.scatter(x, y, s=z/5000, alpha=1)
plt.show()

Um pouquinho de miscelânia agora, vamos descobrir onde os Wal-Mart do Brasil estão, ou pelo menos estavam...

In [ ]:
x = df_01[maskLat&maskLong]["LONG"]
y = df_01[maskLat&maskLong]["LAT"]
z = df_01[maskLat&maskLong]["WAL-MART"]

plt.figure(figsize=(10,10))
plt.title("Longitude x Latitude - Localização dos Wal Mart do Brasil")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.scatter(x, y, s=z, alpha=1)
plt.show()

Vamos criar novas métricas agora. O classificar as cidades que possuem os serviços de Uber, MaCdonald's e Walmart de acordo com as seguintes categorias:
* A -> Cidade possui os três serviços
* B -> Cidade possui Mac e Walmart
* C -> Cidade possui Uber e Walmart
* D -> Cidade possui Uber e Mac
* E -> Cidade possui somente Uber
* F -> Cidade possui somente Mac
* G -> Cidade possui somente walmart
* Z -> Cidade não possui nenhum dos serviços

Para isso vamos iterar o dataset analisando linha a linha para gerar a categoria final.

In [ ]:
cat = []
for i,j in df_01[['UBER', 'MAC', 'WAL-MART']].iterrows():
    if j[0] != 0 and j[1] != 0 and j[2] != 0:
        cat.append('A') # Possui os tres servicos
    elif j[0] == 0 and j[1] != 0 and j[2] != 0:
        cat.append('B') # somente Mac e Walmart
    elif j[0] != 0 and j[1] == 0 and j[2] != 0:
        cat.append('C') # somente Uber e Walmart
    elif j[0] != 0 and j[1] != 0 and j[2] == 0:
        cat.append('D') # somente Uber e Mac
    elif j[0] != 0 and j[1] == 0 and j[2] == 0:
        cat.append('E') # somente Uber
    elif j[0] == 0 and j[1] != 0 and j[2] == 0:
        cat.append('F')
    elif j[0] == 0 and j[1] == 0 and j[2] != 0:
        cat.append('G') # somente walmart
    else:
        cat.append('Z')

df_01.insert(81, 'UMW', cat)

In [ ]:
maskCat = df_01['UMW'] != 0
sns.lmplot(x='LONG',
           y='LAT',
           data=df_01[maskLong&maskLat&maskCat],
           fit_reg=False,
           hue='UMW',
           legend=True,
           scatter_kws={'s':30},
           height=10)

E como isso divide entre os município do país.

In [ ]:
cat = df_01[maskLong&maskLat&maskCat]

tot = cat[['CITY', 'UMW']].groupby('UMW').count().sum().values[0]

labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'Z']
A = np.array(cat[['CITY', 'UMW']].groupby('UMW').count()).reshape(8)/np.array(tot)
plt.figure(figsize=(10,10))
plt.title('Distribuição das categorias')
wedges, texts, autotexts = plt.pie(A, labels=labels, autopct='%.1f%%')
plt.legend(wedges, ['A - Possui os três serviços', 'B - Mac e Walmart', 'C - Uber e Walmart', 'D - Uber e Mac', 'E - Uber', 'F - Mac', 'G - Walmart', 'Z - Nenhum'])
plt.show()

E a qual a parcela da população com acesso a estes serviços ?

In [ ]:
tot = cat[['ESTIMATED_POP', 'UMW']].sum().values[0]
cat[['ESTIMATED_POP', 'UMW']].groupby('UMW').sum()

labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'Z']
A = np.array(cat[['ESTIMATED_POP', 'UMW']].groupby('UMW').sum()).reshape(8)/np.array(tot)
plt.figure(figsize=(10,10))
plt.title('Distribuição das categorias')
wedges, texts, autotexts = plt.pie(A, labels=labels, autopct='%.1f%%')
plt.legend(wedges, ['A - Possui os três serviços', 'B - Mac e Walmart', 'C - Uber e Walmart', 'D - Uber e Mac', 'E - Uber', 'F - Mac', 'G - Walmart', 'Z - Nenhum'])
plt.show()

Vamos explorar um pouco agora o quão sensível algumas variáveis são as outras. Podemos fazer isso por meio de um mapa de calor

In [ ]:
ID01 = df_01[['ESTIMATED_POP', 'AREA', 'IDHM', 'GVA_AGROPEC', 'GVA_INDUSTRY', 'GVA_SERVICES', 'GVA_PUBLIC', 'POP_GDP']]

tot = np.array(ID01['GVA_AGROPEC'])+np.array(ID01['GVA_INDUSTRY'])+np.array(ID01['GVA_SERVICES'])+np.array(ID01['GVA_PUBLIC'])
gva_agro = [j[0]/(j.sum()) if j[0] != 0 else 0 for i,j in ID01[['GVA_AGROPEC', 'GVA_INDUSTRY', 'GVA_SERVICES', 'GVA_PUBLIC']].iterrows()]
gva_ind = [j[1]/(j.sum()) if j[1] != 0 else 0 for i,j in ID01[['GVA_AGROPEC', 'GVA_INDUSTRY', 'GVA_SERVICES', 'GVA_PUBLIC']].iterrows()]
gva_serv = [j[2]/(j.sum()) if j[2] != 0 else 0 for i,j in ID01[['GVA_AGROPEC', 'GVA_INDUSTRY', 'GVA_SERVICES', 'GVA_PUBLIC']].iterrows()]
gva_pub = [j[3]/(j.sum()) if j[3] != 0 else 0 for i,j in ID01[['GVA_AGROPEC', 'GVA_INDUSTRY', 'GVA_SERVICES', 'GVA_PUBLIC']].iterrows()]

ID01.insert(8, 'GVA_AGROPEC_%', gva_agro)
ID01.insert(9, 'GVA_INDUSTRY_%', gva_ind)
ID01.insert(10, 'GVA_SERVICES_%', gva_serv)
ID01.insert(11, 'GVA_PUBLIC_%', gva_pub)

corrmat = ID01.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
#plot heat map
g=sns.heatmap(ID01[top_corr_features].corr(),annot=True,cmap="RdYlGn")

Parece que existe uma correlação forte entre algumas variáveis, logo podemos montar um modelo de regressão linear para entendermos como um variável se comporta em função da outra.

Para isso vamos precisar de mais algumas bibliotecas para que possamos cumprir essa missão.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

Vamos deixar bem separado nosso domínio e contra domínio

In [ ]:
X = ID01[['GVA_INDUSTRY', 'GVA_SERVICES', 'GVA_PUBLIC']]
Y = ID01[['ESTIMATED_POP']]

Agora vamos separa o conjuntos de dados em dois, um para treinarmento e outro para teste do modelo criado

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Vamos tentar uma fução que descreve bem os dados. A priemira será a regresão linear: LinearRegression

In [ ]:
reg = LinearRegression()

Vamos alimetar o modelo com dados obter os coeficienes para cada eixo.

In [ ]:
reg.fit(X_train, Y_train)

In [ ]:
print('Score(R2) do modelo para o conjunto de treino: {}'.format(reg.score(X_train,Y_train)))

In [ ]:
reg.coef_

In [ ]:
reg.intercept_

Desta forms a equação fica $$f(x,y,z) = cof1x + cof2y + cof3z + int$$

In [ ]:
print('f(x,y,z) = {}x + {}y + {}z + {}'.format(reg.coef_[0][0], reg.coef_[0][1], reg.coef_[0][2], reg.intercept_[0]))

Com isso descobrimos a melhor função que descreve os dados obtidos para o modelo de LinearRegression

In [ ]:
pred = reg.predict(X_test)

In [ ]:
print('Erro quadrado médio: {}'.format(mean_squared_error(Y_test, pred)))
print('R2 real: {}'.format(r2_score(Y_test, pred)))

Vamos testar agora uma regressão com SVM

In [ ]:
from sklearn import svm

In [ ]:
clf = svm.SVR()

In [ ]:
clf.fit(X_train, Y_train)

In [ ]:
pred_svm = clf.predict(X_test)

In [ ]:
print('Erro quadrado médio: {}'.format(mean_squared_error(Y_test, pred_svm)))

In [ ]:
print('R2 real: {}'.format(r2_score(Y_test, pred_svm)))

Agora vamos testar uma com Árvore de Decisão

In [ ]:
from sklearn import tree

In [ ]:
arv = tree.DecisionTreeRegressor()

In [ ]:
arv.fit(X_train, Y_train)

In [ ]:
pred_arv = arv.predict(X_test)

In [ ]:
print('Erro quadrado médio: {}'.format(mean_squared_error(Y_test, pred_arv)))
print('R2 real: {}'.format(r2_score(Y_test, pred_arv)))
print(Y_test.iloc[293].values, arv.predict([X_test.iloc[293].values]))

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [ ]:
# Credenciais do Watson Machine Learning
wml_credentials  = {
  "apikey": "Picture yourself in a boat on a river",
  "iam_apikey_description": "With tangerine trees and marmalade skies",
  "iam_apikey_name": "Somebody calls you, you answer quite slowly",
  "iam_role_crn": "A girl with kaleidoscope eyes",
  "iam_serviceid_crn": "Cellophane flowers of yellow and green",
  "instance_id": "Towering over your head",
  "password": "Look for the girl with the sun in her eyes",
  "url": "And she's gone",
  "username": "Lucy in the sky with diamonds"
}
client = WatsonMachineLearningAPIClient( wml_credentials )

# Definição de metadados do modelo (versao de python, framework, libs e etc)
metadata = {
    client.repository.ModelMetaNames.NAME              : 'Intro Data Science',
    client.repository.ModelMetaNames.FRAMEWORK_NAME    : 'scikit-learn',
    client.repository.ModelMetaNames.FRAMEWORK_VERSION : '0.19',
    client.repository.ModelMetaNames.RUNTIME_NAME      : 'python',
    client.repository.ModelMetaNames.RUNTIME_VERSION   : '3.6'
}


# Conexão com o WML
model_details = client.repository.store_model( reg, meta_props=metadata, training_data=None )

# Deploy do modelo
model_id = model_details["metadata"]["guid"]
model_deployment_details = client.deployments.create( artifact_uid=model_id, name="Ha" )

# Retrieve da URL da API para consumo da mesma
model_endpoint_url = client.deployments.get_scoring_url( model_deployment_details )
print("A URL de chamada da sua API é : ",model_endpoint_url)

### Referências 


- [Dataset](https://www.kaggle.com/crisparada/brazilian-cities/version/7)
- [Notebook guia](https://www.kaggle.com/crisparada/brazilian-cities-a-simple-exploration)
- [Pandas](https://pandas.pydata.org/)
- [Numpy](https://numpy.org/)
- [Matplotlib](https://matplotlib.org/)
- [Seaborn](https://seaborn.pydata.org/)
- [Scikit-Learn](https://scikit-learn.org/stable/)

<h1 style='text-align:center'>Obrigado!<h1>